In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

# Set working directory
os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")

# Import data
data = pd.read_excel("2024 10 16 - Base Original.xlsx")

# Prepare data
def prepare_data(df, columns):
    return df[columns].applymap(lambda x: 1 if x in ["Si", np.nan] else 0)

tip_usu = prepare_data(data, ["Emprendedores", "Mipymes", "Grandes empresas", "Gobierno ", "Academia", "Entidades de soporte", "Personas naturales"])
tip_apoyo = prepare_data(data, ["Formación de Talento Humano", "Tipo de Sistemas de Información", "Asistencia técnica y Consultoría", "Tipo de Redes de Colaboración", "Apoyo Financiero", "Incentivos Tributarios", "Eventos", "Compra Pública", "Bonos o Vouchers", "Premios y Reconocimientos", "Instrumentos Regulatorios"])
ob_oferta = prepare_data(data, ["Innovación", "Emprendimiento", "Economía Popular", "Transferencia de Conocimiento y Tecnología", "Investigación", "Formación de Capital Humano", "Propiedad intelectual", "Calidad", "Clúster o encadenamientos", "Financiación", "Comercialización", "Formalización", "Crecimiento Sostenible", "Inclusión Financiera", "Comercio Electrónico", "Educación económica y financiera", "Competencias Digitales"])

# Custom cosine similarity function
def custom_cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    magnitude_a = np.linalg.norm(a)
    magnitude_b = np.linalg.norm(b)
    
    if magnitude_a == 0 or magnitude_b == 0:
        return 0
    else:
        return dot_product / (magnitude_a * magnitude_b)

# Function to calculate detailed cosine similarity
def calculate_detailed_cosine_similarity(df):
    n, m = df.shape
    result = []
    
    for i in range(n):
        for j in range(n):
            if i != j:
                cosine_sim = custom_cosine_similarity(df.iloc[i], df.iloc[j]) * 100
                element_sim = (df.iloc[i] == df.iloc[j]) * 100
                
                result.append({
                    "Instrumento1": data["Nombre del Instrumento"].iloc[i],
                    "Instrumento2": data["Nombre del Instrumento"].iloc[j],
                    "Similitud_Coseno": cosine_sim,
                    **{col: sim for col, sim in zip(df.columns, element_sim)}
                })
    
    return pd.DataFrame(result)

# Calculate detailed cosine similarities
sim_usu = calculate_detailed_cosine_similarity(tip_usu)
sim_apoyo = calculate_detailed_cosine_similarity(tip_apoyo)
sim_oferta = calculate_detailed_cosine_similarity(ob_oferta)

# Function to export to Excel
def export_to_excel(df, filename):
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Similitud', index=False)

# Export results
export_to_excel(sim_usu, "Similitud_coseno_desagregada_usuarios_1.xlsx")
export_to_excel(sim_apoyo, "Similitud_coseno_desagregada_apoyo_1.xlsx")
export_to_excel(sim_oferta, "Similitud_coseno_desagregada_oferta_1.xlsx")

C:\Users\sebas\AppData\Local\Temp\ipykernel_11540\2977026901.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df[columns].applymap(lambda x: 1 if x in ["Si", np.nan] else 0)
C:\Users\sebas\AppData\Local\Temp\ipykernel_11540\2977026901.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df[columns].applymap(lambda x: 1 if x in ["Si", np.nan] else 0)
C:\Users\sebas\AppData\Local\Temp\ipykernel_11540\2977026901.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df[columns].applymap(lambda x: 1 if x in ["Si", np.nan] else 0)


In [12]:
import pandas as pd
import numpy as np
import os

# Establecer el directorio de trabajo
os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")

# Cargar los datos
data = pd.read_excel("2024 10 17 - Base Original.xlsx")

def prepare_data(df, columns):
    return df[columns].replace({"Si": 1, "No": 0}).fillna(0)

def calculate_similarity(row1, row2):
    # Convertir a arrays booleanos para facilitar operaciones de conjuntos
    set1 = row1.astype(bool)
    set2 = row2.astype(bool)
    
    # Calcular intersección y unión
    intersection = np.logical_and(set1, set2).sum()
    union = np.logical_or(set1, set2).sum()
    
    # Evitar división por cero
    if union == 0:
        return 0
    
    # Calcular índice de Jaccard y convertir a porcentaje
    return (intersection / union) * 100

def categorize_similarity(value, thresholds):
    for i, threshold in enumerate(thresholds):
        if value <= threshold:
            return i
    return len(thresholds)

def analyze_articulation(data):
    results = []
    
    usuarios_cols = ['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 'Entidades de soporte', 'Personas naturales']
    objetivo_cols = ['Innovación', 'Emprendimiento', 'Economía Popular', 'Transferencia de Conocimiento y Tecnología', 'Investigación', 'Formación de Capital Humano', 'Propiedad intelectual', 'Calidad', 'Clúster o encadenamientos', 'Financiación', 'Comercialización', 'Formalización', 'Crecimiento Sostenible', 'Inclusión Financiera', 'Comercio Electrónico', 'Educación económica y financiera', 'Competencias Digitales']
    apoyo_cols = ['Formación de Talento Humano', 'Tipo de Sistemas de Información', 'Asistencia técnica y Consultoría', 'Tipo de Redes de Colaboración', 'Apoyo Financiero', 'Incentivos Tributarios', 'Eventos', 'Compra Pública', 'Bonos o Vouchers', 'Premios y Reconocimientos', 'Instrumentos Regulatorios']
    
    prepared_data = prepare_data(data, usuarios_cols + objetivo_cols + apoyo_cols)
    
    all_similarities = {
        'USUARIOS': [],
        'OBJETIVO': [],
        'APOYO': [],
        'ENTIDAD': []
    }
    
    # Calcular todas las similitudes primero para establecer los umbrales
    for i, row in prepared_data.iterrows():
        for j, other_row in prepared_data.iterrows():
            if i != j:
                all_similarities['USUARIOS'].append(calculate_similarity(row[usuarios_cols], other_row[usuarios_cols]))
                all_similarities['OBJETIVO'].append(calculate_similarity(row[objetivo_cols], other_row[objetivo_cols]))
                all_similarities['APOYO'].append(calculate_similarity(row[apoyo_cols], other_row[apoyo_cols]))
                all_similarities['ENTIDAD'].append(100 if data.loc[i, 'Entidad'] == data.loc[j, 'Entidad'] else 0)
    
    thresholds = {k: np.percentile(v, [25, 50, 75]) for k, v in all_similarities.items()}
    
    # Analizar cada instrumento
    for i, row in prepared_data.iterrows():
        similarities = []
        
        for j, other_row in prepared_data.iterrows():
            if i != j:
                sim = {
                    'USUARIOS': calculate_similarity(row[usuarios_cols], other_row[usuarios_cols]),
                    'OBJETIVO': calculate_similarity(row[objetivo_cols], other_row[objetivo_cols]),
                    'APOYO': calculate_similarity(row[apoyo_cols], other_row[apoyo_cols]),
                    'ENTIDAD': 100 if data.loc[i, 'Entidad'] == data.loc[j, 'Entidad'] else 0
                }
                sim['TOTAL'] = np.mean(list(sim.values()))
                sim['index'] = j
                similarities.append(sim)
        
        similarities.sort(key=lambda x: x['TOTAL'], reverse=True)
        best_match = similarities[0]
        
        avg_similarities = {k: best_match[k] for k in ['USUARIOS', 'OBJETIVO', 'APOYO', 'ENTIDAD']}
        similarity_categories = {k: categorize_similarity(v, thresholds[k]) for k, v in avg_similarities.items()}
        
        # Determinar nivel de articulación
        if all(cat == 3 for cat in similarity_categories.values()):
            articulation_level = "2. Fusionar"
        elif similarity_categories['USUARIOS'] >= 2 and similarity_categories['OBJETIVO'] >= 2 and similarity_categories['APOYO'] >= 2:
            articulation_level = "3. Definir roles"
        elif similarity_categories['USUARIOS'] >= 2 and similarity_categories['OBJETIVO'] >= 2:
            articulation_level = "4. Empaquetar"
        elif similarity_categories['USUARIOS'] >= 2:
            articulation_level = "5. Generar rutas"
        else:
            articulation_level = "1. Renunciar"
        
        # Guardar resultados
        results.append({
            'Instrumento1': data.loc[i, 'Nombre del Instrumento'],
            'Descripción1': data.loc[i, 'Descripción'],
            'Entidad1': data.loc[i, 'Entidad'],
            'Instrumento2': data.loc[best_match['index'], 'Nombre del Instrumento'],
            'Descripción2': data.loc[best_match['index'], 'Descripción'],
            'Entidad2': data.loc[best_match['index'], 'Entidad'],
            'Nivel de Articulación': articulation_level,
            'Similitud Total': best_match['TOTAL'],
            **avg_similarities,
            **{f"{k}_Categoría": v for k, v in similarity_categories.items()}
        })
    
    return pd.DataFrame(results)

# Ejecutar el análisis
articulation_results = analyze_articulation(data)

# Exportar resultados
articulation_results.to_excel("Analisis_Articulacion_Instrumentos_Detallado.xlsx", index=False)

print("Análisis completado. Los resultados se han guardado en 'Analisis_Articulacion_Instrumentos_Detallado.xlsx'.")

KeyboardInterrupt: 

## Con distancia de Hamming

In [13]:
import pandas as pd
import numpy as np
import os
from scipy.spatial.distance import cdist

# Establecer el directorio de trabajo
os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")

# Cargar los datos
data = pd.read_excel("2024 10 17 - Base Original.xlsx")

def prepare_data(df, columns):
    return df[columns].replace({"Si": 1, "No": 0}).fillna(0).values

def calculate_similarity_matrix(matrix1, matrix2):
    """
    Calcula la matriz de similitud usando operaciones vectorizadas.
    Retorna porcentajes de similitud (100 - distancia_hamming).
    """
    # Calcula la distancia de Hamming normalizada entre todas las filas
    distances = cdist(matrix1, matrix2, metric='hamming')
    # Convierte distancias a similitudes (en porcentaje)
    return (1 - distances) * 100

def analyze_articulation(data):
    # Definir columnas para cada categoría
    usuarios_cols = ['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 'Entidades de soporte', 'Personas naturales']
    objetivo_cols = ['Innovación', 'Emprendimiento', 'Economía Popular', 'Transferencia de Conocimiento y Tecnología', 'Investigación', 'Formación de Capital Humano', 'Propiedad intelectual', 'Calidad', 'Clúster o encadenamientos', 'Financiación', 'Comercialización', 'Formalización', 'Crecimiento Sostenible', 'Inclusión Financiera', 'Comercio Electrónico', 'Educación económica y financiera', 'Competencias Digitales']
    apoyo_cols = ['Formación de Talento Humano', 'Tipo de Sistemas de Información', 'Asistencia técnica y Consultoría', 'Tipo de Redes de Colaboración', 'Apoyo Financiero', 'Incentivos Tributarios', 'Eventos', 'Compra Pública', 'Bonos o Vouchers', 'Premios y Reconocimientos', 'Instrumentos Regulatorios']
    
    # Preparar matrices binarias
    usuarios_matrix = prepare_data(data, usuarios_cols)
    objetivo_matrix = prepare_data(data, objetivo_cols)
    apoyo_matrix = prepare_data(data, apoyo_cols)
    
    # Calcular matrices de similitud para cada categoría
    usuarios_sim = calculate_similarity_matrix(usuarios_matrix, usuarios_matrix)
    objetivo_sim = calculate_similarity_matrix(objetivo_matrix, objetivo_matrix)
    apoyo_sim = calculate_similarity_matrix(apoyo_matrix, apoyo_matrix)
    
    # Crear matriz de similitud para entidades
    n = len(data)
    entidad_sim = np.zeros((n, n))
    entidades = data['Entidad'].values
    entidad_sim = np.equal.outer(entidades, entidades).astype(float) * 100
    
    # Calcular umbrales para cada categoría
    def calculate_thresholds(sim_matrix):
        # Obtener valores superiores de la matriz triangular (excluyendo la diagonal)
        upper_tri = sim_matrix[np.triu_indices(n, k=1)]
        return np.percentile(upper_tri, [25, 50, 75])
    
    thresholds = {
        'USUARIOS': calculate_thresholds(usuarios_sim),
        'OBJETIVO': calculate_thresholds(objetivo_sim),
        'APOYO': calculate_thresholds(apoyo_sim),
        'ENTIDAD': calculate_thresholds(entidad_sim)
    }
    
    results = []
    
    # Analizar cada instrumento
    for i in range(n):
        # Calcular similitud total promedio con todos los demás instrumentos
        total_sim = (usuarios_sim[i] + objetivo_sim[i] + apoyo_sim[i] + entidad_sim[i]) / 4
        total_sim[i] = -1  # Excluir comparación consigo mismo
        
        # Encontrar el mejor match
        best_match_idx = np.argmax(total_sim)
        
        # Obtener similitudes individuales con el mejor match
        similarities = {
            'USUARIOS': usuarios_sim[i, best_match_idx],
            'OBJETIVO': objetivo_sim[i, best_match_idx],
            'APOYO': apoyo_sim[i, best_match_idx],
            'ENTIDAD': entidad_sim[i, best_match_idx]
        }
        
        # Categorizar similitudes
        similarity_categories = {
            k: sum(v > threshold for threshold in thresholds[k])
            for k, v in similarities.items()
        }
        
        # Determinar nivel de articulación
        if all(cat == 3 for cat in similarity_categories.values()):
            articulation_level = "2. Fusionar"
        elif similarity_categories['USUARIOS'] >= 2 and similarity_categories['OBJETIVO'] >= 2 and similarity_categories['APOYO'] >= 2:
            articulation_level = "3. Definir roles"
        elif similarity_categories['USUARIOS'] >= 2 and similarity_categories['OBJETIVO'] >= 2:
            articulation_level = "4. Empaquetar"
        elif similarity_categories['USUARIOS'] >= 2:
            articulation_level = "5. Generar rutas"
        else:
            articulation_level = "1. Renunciar"
        
        # Guardar resultados
        results.append({
            'Instrumento1': data.iloc[i]['Nombre del Instrumento'],
            'Descripción1': data.iloc[i]['Descripción'],
            'Entidad1': data.iloc[i]['Entidad'],
            'Instrumento2': data.iloc[best_match_idx]['Nombre del Instrumento'],
            'Descripción2': data.iloc[best_match_idx]['Descripción'],
            'Entidad2': data.iloc[best_match_idx]['Entidad'],
            'Nivel de Articulación': articulation_level,
            'Similitud Total': total_sim[best_match_idx],
            **similarities,
            **{f"{k}_Categoría": v for k, v in similarity_categories.items()}
        })
    
    return pd.DataFrame(results)

# Ejecutar el análisis
articulation_results = analyze_articulation(data)

# Exportar resultados
articulation_results.to_excel("Analisis_Articulacion_Instrumentos_Detallado.xlsx", index=False)

print("Análisis completado. Los resultados se han guardado en 'Analisis_Articulacion_Instrumentos_Detallado.xlsx'.")

Análisis completado. Los resultados se han guardado en 'Analisis_Articulacion_Instrumentos_Detallado.xlsx'.
